In [3]:
### header ###
__author__ = "Jenhan Tao"
__license__ = "MIT"
__email__ = "jenhantao@gmail.com"

### imports ###
import sys # system functions
import os # os functions
import pandas as pd # for reading data
import numpy as np # for numeric operations
import matplotlib.pyplot as plt 
import matplotlib # for visualizing data
import scipy # scientific computing
import seaborn as sns # for pretty plots

### notebook specific configuration ###
%matplotlib inline
matplotlib.rcParams['savefig.dpi'] = 200
sns.set_context('notebook')
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# it's bad practice to work within a github repo and also bad practice to have data in a github repo
# but it's convenient
# change working directory to github repo
working_directory = '/gpfs/data01/glasslab/home/jtao/side_projects/fantasy_football_genetic_algorithm_draft/'
os.chdir(working_directory)

### Read in rankings

In [17]:
# year_rankFrame_dict = {} # {year:DataFrame}
# for f in os.listdir(working_directory + '/rankings_ffc_standard'):
#     json_path = working_directory + '/rankings_ffc_standard/' + f
#     ranks = pd.read_json(json_path, typ='series')
#     adp_list = [x['adp'] for x in ranks['players']]
#     name_list = [x['name'] for x in ranks['players']]
#     position_list = [x['position'] for x in ranks['players']]
    
#     year = f.split('_')[-1].replace('.json','')
    
#     rankFrame = pd.DataFrame({'ADP':adp_list, 'Position':position_list, 'Name':name_list})
#     year_rankFrame_dict[year] = rankFrame
    


In [168]:
year_rankFrame_dict = {} # {year:DataFrame}
for f in os.listdir(working_directory + '/rankings_mfl_standard'):
    csv_path = working_directory + '/rankings_mfl_standard/' + f
    rankFrame = pd.read_csv(csv_path, sep=',')
    rankFrame['POSITION'] = [x.split(' ')[-1].replace('*','') for x in rankFrame['PLAYER'].values]
    rankFrame['NAME'] = [' '.join(x.split(', ')[1].split(' ')[:-2]) + ' ' + x.split(', ')[0].replace(',','') for x in rankFrame['PLAYER'].values]
    
    rankFrame = rankFrame[rankFrame['POSITION'].isin(['Def', 'PK', 'QB', 'RB', 'TE', 'WR'])]
    rankFrame['ADP SCORE'] = rankFrame['AVG. PICK'].max() - rankFrame['AVG. PICK'].values
    year = f.split('_')[-1].replace('.csv','')
    
    year_rankFrame_dict[year] = rankFrame
    


### Read in season results

In [123]:
year_scoreFrame_dict = {} # {year:DataFrame}
for f in os.listdir(working_directory + '/scoring_standard'):
    score_path = working_directory + '/scoring_standard/' + f
    scoreFrame = pd.read_csv(score_path, sep=',')
    year = f.split('_')[-1].replace('.csv','')
    year_scoreFrame_dict[year] = scoreFrame
    


In [124]:
scoreFrame.head()

,Player,Points
0,Stephen Gostkowski,159
1,Graham Gano,150
2,Blair Walsh,147
3,Robbie Gould,141
4,Josh Brown,140


### Create Players

In [152]:
position_index_dict = dict(zip(sorted(rankFrame['POSITION'].unique()), 
                               range(len(rankFrame['POSITION'].unique()))))
index_position_dict = dict(zip(range(len(rankFrame['POSITION'].unique())),
                                sorted(rankFrame['POSITION'].unique()), 
                               ))

In [153]:
position_index_dict # gives the position and the corresponding index

{'DE': 0,
 'Def': 1,
 'LB': 2,
 'PK': 3,
 'QB': 4,
 'RB': 5,
 'S': 6,
 'TE': 7,
 'WR': 8}

In [154]:
index_position_dict # gives the index and the corresponding index

{0: 'DE',
 1: 'Def',
 2: 'LB',
 3: 'PK',
 4: 'QB',
 5: 'RB',
 6: 'S',
 7: 'TE',
 8: 'WR'}

In [128]:
def create_new_players(num_players, num_rounds):
    '''
    inputs: number of players and rounds
    outputs: list of fantasy players
    '''
    num_positions = 6 # number of positions
    player_list = []
    for player_number in range(num_players):
        new_player = []
        for round_number in range(num_rounds):
            weights = np.random.random(num_positions) # initialize random weights
            normed_weights = weights/np.sum(weights) # normalize so weights sum to 1
            new_player.append(normed_weights)
    
        player_list.append((new_player))
    return np.array(player_list)

In [151]:
len(rankFrame['POSITION'].unique()) # number of positions

9

In [40]:
fantasy_players = create_new_players(12, 10)

In [41]:
fantasy_players.shape # first index gives the player
                      # second index gives the round
                      # third index gives the position 


(12, 10, 6)

In [ ]:
fantasy_players[0,0,0] #gives the importance for player 1 in the first round for a defense 
fantasy_players[0,0,1] #gives the importance for player 1 in the first round for a place kicker (PK)
fantasy_players[8,3,2] #gives the importance for player 9 in the fourth round for a QB

In [53]:
fantasy_players.shape

(12, 10, 6)

### Run Simulation

In [157]:
rankFrame = year_rankFrame_dict['2017']

scoreFrame = year_scoreFrame_dict['2017']

In [158]:
rankFrame.shape

(340, 9)

In [159]:
scoreFrame.shape

(461, 2)

In [160]:
score_players = set (scoreFrame['Player'].values)

In [161]:
rank_players = set(rankFrame['NAME'].values)

In [165]:
len(score_players - rank_players)

186

In [164]:
len(rank_players-score_players)

65

In [166]:
def simulate_draft(rankFrame, fantasy_players, num_rounds):
    '''
    returns a list of rosters for each fantasy player
    '''
    position_index_dict
    drafted_players = set()
    for draft_round in range(num_rounds):
        for player in range(fantasy_players.shape[0]):
            # retrieve strategy
            round_strategy = fantasy_players[player,draft_round, :]
            # calculate player ranks
            print(round_strategy)
            break
    return fantasy_teams

In [167]:
simulate_draft(rankFrame, fantasy_players, 10)

[0.12450951 0.24221658 0.22053774 0.201553   0.10757643 0.10360674]
[0.00133977 0.27464183 0.13641913 0.18885906 0.22736425 0.17137595]
[0.02289191 0.16397377 0.22024126 0.14538954 0.2077734  0.23973012]
[0.11976985 0.42714667 0.24149384 0.10378072 0.09092644 0.01688249]
[0.1010647  0.19731403 0.16094528 0.22204605 0.09638806 0.22224188]
[0.05590035 0.05284243 0.14348468 0.14838749 0.27389808 0.32548698]
[0.05243324 0.28215605 0.25429574 0.12486966 0.16162869 0.12461661]
[0.25929737 0.12584274 0.05952572 0.23525717 0.11059225 0.20948476]
[0.10221644 0.13809797 0.02200412 0.20217354 0.21543265 0.32007528]
[0.32727354 0.07096239 0.11728432 0.27294256 0.00364651 0.20789067]


NameError: name 'fantasy_teams' is not defined

In [ ]:
def score_roster(score_frame, fantasy_teams):
    '''
    returns a list of performances for each fantasy player's team
    '''
    
    return fantasy_team_performances